In [1]:
!pip install sounddevice
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.2.2-2.1ubuntu2.5).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 188 kB of archives.
After this operation, 926 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudiocpp0 amd64 19.6.0-1build1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 portaudio19-dev amd64 19.6.0-1build1 [106 kB]
Fetched 188 kB in 1s (202 kB/s)
Selecting previously unselected

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import sounddevice as sd
import soundfile as sf
import pyaudio
import librosa
import IPython.display as ipd

from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [11]:
#Listen

import numpy
sr = 22050 # sample rate
my_array = np.load("/content/drive/MyDrive/audio_recreation[VQVAE(7K)]/000620.npy")

# Do something with my_array
print(my_array)


import IPython.display as ipd

ipd.Audio(my_array, rate=sr) # load a NumPy array

[-0.0003075  -0.00064988 -0.00085367 ...  0.00139146  0.00168485
  0.00206107]


Griffin & Lim algorithm Function

In [3]:
def get_windowed_signal(recovered_spectrum, hop_size, fft_size, signal_len, opt_win = 'hann'):
    assert fft_size == int(fft_size)
    assert hop_size == int(hop_size)
    
    if opt_win == 'hann':
        window = np.hanning(fft_size);
    else:
        raise Exception("Unkown window.")
    
    n_frames = recovered_spectrum.shape[1]
    
    proposed_signal = np.zeros(signal_len)
    window_acc = np.zeros(signal_len)
    
    for frame in range(n_frames):
        #print("Frame {0}".format(frame))
        
        begin = frame*(hop_size)
        end = begin + fft_size
        proposed_signal[begin:end] += window*np.real(np.fft.ifft(recovered_spectrum[:,frame],fft_size))
        
    return proposed_signal

def get_distance(X, Y, Dprev = None, thresh = None):
    diff = (np.abs(X) - np.abs(Y))**2
    #print("X: {0}; Y: {1}; Difference: {2}".format(X,Y,diff))
    D = np.sum(diff)/diff.size
    
    if not Dprev:
        return D, False
    
    # Evaluate Euclidian distance:
    if (Dprev - D)/Dprev < thresh:
        return D, True
    
    return D, False

def save_signal(y, file, sr):
    sf.write(file, y, sr)


def run_GL(MAG_spec, sr, fft_size, hop_size, output_file, window = 'hann'):
    
    # Minimum relative distance threshold:
    thresh = 0.01
    
    sig_len = MAG_spec.shape[1]*hop_size + (fft_size - hop_size)
    
    initial_spectrum = MAG_spec #Yw
    
    recovered_spectrum = np.abs(initial_spectrum)*np.exp(1j*0)
    
    opt_win = 'hann'
    
    proposed_signal = get_windowed_signal(recovered_spectrum, hop_size,
                                          fft_size, sig_len, opt_win)
    
    proposed_spectrum = librosa.stft(proposed_signal, n_fft=fft_size, 
                                     hop_length=hop_size, window=window, 
                                     center=False, pad_mode='reflect') 
    
    D, move_on = get_distance(recovered_spectrum, proposed_spectrum)
    
    iteration = 0
    
    while not move_on:
        
        iteration += 1
        
        recovered_spectrum = np.abs(initial_spectrum)*np.exp(1j*np.angle(proposed_spectrum))
        
        proposed_signal = get_windowed_signal(recovered_spectrum, hop_size,
                                              fft_size, sig_len, opt_win)
        
        proposed_spectrum = librosa.stft(proposed_signal, n_fft=fft_size, 
                                         hop_length=hop_size, window='hann', 
                                         center=False, pad_mode='reflect')
        
        D, move_on = get_distance(recovered_spectrum, proposed_spectrum, D, thresh)        
        
    
    to_be_saved = proposed_signal
    save_signal(to_be_saved, output_file, sr)
    
    print("Griffin & Lim Construction done")

Driver

In [4]:
def main(filename):    
    fft_size = 2048
    hop_size = fft_size//4
    win = 'hann'

    signal, sr = np.load(filename), 22050

    MAG_spec = librosa.stft(signal, n_fft=fft_size, 
                            hop_length=hop_size, window=win, 
                            center=False, pad_mode='reflect')

    run_GL(MAG_spec, sr, fft_size, hop_size, window = win, output_file = filename.split("/")[-1].split(".")[0] + ".wav")

In [12]:
filename = "/content/drive/MyDrive/audio_recreation[VQVAE(7K)]/000620.npy"
main(filename)

Griffin & Lim Construction done


In [13]:
y, sr = librosa.load(filename.split("/")[-1].split(".")[0] + ".wav")
ipd.Audio(y, rate=sr)